In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import nltk
nltk.download('punkt')
nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
import json
import pickle

import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from keras.optimizers import SGD
import random

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [ ]:
words=[]
classes = []
documents = []
ignore_words = ['?', '!']
data_file = open('/content/drive/MyDrive/Simple-Python-Chatbot-master/Simple-Python-Chatbot-master/intents_kriya.json').read()
intents = json.loads(data_file)


for intent in intents['intents']:
    for pattern in intent['patterns']:

        # take each word and tokenize it
        w = nltk.word_tokenize(pattern)
        words.extend(w)
        # adding documents
        documents.append((w, intent['tag']))

        # adding classes to our class list
        if intent['tag'] not in classes:
            classes.append(intent['tag'])

words = [lemmatizer.lemmatize(w.lower()) for w in words if w not in ignore_words]
words = sorted(list(set(words)))

classes = sorted(list(set(classes)))

print (len(documents), "documents")

print (len(classes), "classes", classes)

print (len(words), "unique lemmatized words", words)


pickle.dump(words,open('/content/drive/MyDrive/Simple-Python-Chatbot-master/Simple-Python-Chatbot-master/words3.pkl','wb'))
pickle.dump(classes,open('/content/drive/MyDrive/Simple-Python-Chatbot-master/Simple-Python-Chatbot-master/classes3.pkl','wb'))

# initializing training data
training = []
output_empty = [0] * len(classes)
for doc in documents:
    # initializing bag of words
    bag = []
    # list of tokenized words for the pattern
    pattern_words = doc[0]
    # lemmatize each word - create base word, in attempt to represent related words
    pattern_words = [lemmatizer.lemmatize(word.lower()) for word in pattern_words]
    # create our bag of words array with 1, if word match found in current pattern
    for w in words:
        bag.append(1) if w in pattern_words else bag.append(0)

    # output is a '0' for each tag and '1' for current tag (for each pattern)
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1

    training.append([bag, output_row])
# shuffle our features and turn into np.array
random.shuffle(training)
training = np.array(training)
# create train and test lists. X - patterns, Y - intents
train_x = list(training[:,0])
train_y = list(training[:,1])
print("Training data created")


# Create model - 3 layers. First layer 128 neurons, second layer 64 neurons and 3rd output layer contains number of neurons
# equal to number of intents to predict output intent with softmax
model = Sequential()
model.add(Dense(128, input_shape=(len(train_x[0]),), activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]), activation='softmax'))

# Compile model. Stochastic gradient descent with Nesterov accelerated gradient gives good results for this model
sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

#fitting and saving the model
hist = model.fit(np.array(train_x), np.array(train_y), epochs=1000, batch_size=5, verbose=1)
model.save('/content/drive/MyDrive/Simple-Python-Chatbot-master/Simple-Python-Chatbot-master/kriya_chatbot_model.h5', hist)

print("model created")


331 documents
35 classes ['Affiliation', 'Bold', 'Bullet list', 'Decrease Indent', 'Hide track changes', 'Increase Indent', 'Insert special character', 'Merge with next', 'Merge with previous', 'Move Down', 'Move up', 'Number list', 'Place float', 'Place inline', 'Redo', 'Show track changes', 'Spell check', 'Undo', 'Upload Offline PDF', 'article type', 'background color', 'change case', 'copy', 'corresponding author', 'cut', 'goodbye', 'greeting', 'italic', 'options', 'section head', 'subscript', 'superscript', 'text color', 'thanks', 'underline']
354 unique lemmatized words ["'m", "'s", "'ve", ',', 'a', 'academic', 'accidentally', 'add', 'adding', 'adjust', 'adjusting', 'affiliation', 'after', 'already', 'altering', 'am', 'amount', 'an', 'and', 'another', 'any', 'anyone', 'appear', 'appearance', 'appears', 'application', 'applied', 'apply', 'applying', 'are', 'article', 'at', 'author', 'available', 'awesome', 'back', 'background', 'be', 'been', 'before', 'being', 'best', 'between', 'b

<ipython-input-7-28f2104fcca9>:58: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  training = np.array(training)


Training data created


/usr/local/lib/python3.9/dist-packages/keras/optimizers/optimizer_v2/gradient_descent.py:114: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


Epoch 1/200
67/67 [==============================] - 1s 3ms/step - loss: 3.5905 - accuracy: 0.0242
Epoch 2/200
67/67 [==============================] - 0s 3ms/step - loss: 3.4671 - accuracy: 0.0755
Epoch 3/200
67/67 [==============================] - 0s 2ms/step - loss: 3.3085 - accuracy: 0.1118
Epoch 4/200
67/67 [==============================] - 0s 2ms/step - loss: 3.0945 - accuracy: 0.1631
Epoch 5/200
67/67 [==============================] - 0s 2ms/step - loss: 2.8516 - accuracy: 0.1903
Epoch 6/200
67/67 [==============================] - 0s 2ms/step - loss: 2.5875 - accuracy: 0.2326
Epoch 7/200
67/67 [==============================] - 0s 2ms/step - loss: 2.2958 - accuracy: 0.2840
Epoch 8/200
67/67 [==============================] - 0s 2ms/step - loss: 2.1083 - accuracy: 0.3535
Epoch 9/200
67/67 [==============================] - 0s 2ms/step - loss: 1.9403 - accuracy: 0.3807
Epoch 10/200
67/67 [==============================] - 0s 2ms/step - loss: 1.8138 - accuracy: 0.4018
Epoch 11/